In [ ]:
import tensorflow as tf

In [ ]:
daily_sales_numbers = [19, 56, -69, 36, 72, 101, -18, -55, 420, 365]

In [ ]:
# converting python list into tf dataset where each value is converted into a tensor
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

In [ ]:
for x in tf_dataset.as_numpy_iterator():
  print(x)

19
56
-69
36
72
101
-18
-55
420
365


In [ ]:
# getting rid of invalid/negative data points
tf_dataset = tf_dataset.filter(lambda x:x>0)
for x in tf_dataset.as_numpy_iterator():
  print(x)

19
56
36
72
101
420
365


In [ ]:
#  converting USD in PKR
tf_dataset = tf_dataset.map(lambda x:x*288)
for x in tf_dataset.as_numpy_iterator():
  print(x)

5472
16128
10368
20736
29088
120960
105120


In [ ]:
# shuffling the dataset with buffer size: n where buffer is a selection of random values of number n and from these values a random value is picked
tf_dataset = tf_dataset.shuffle(3)
for x in tf_dataset.as_numpy_iterator():
  print(x)

16128
5472
29088
20736
105120
10368
120960


In [ ]:
# creating batches of size n
for sales_batch in tf_dataset.batch(2):
  print(sales_batch.numpy())

[ 5472 10368]
[20736 16128]
[120960 105120]
[29088]


In [ ]:
# creating dataset again and combining all methods into one single line
tf_dataset2 = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset2 = tf_dataset2.filter(lambda x:x>0).map(lambda y:y*288).shuffle(3).batch(2)
for x in tf_dataset2.as_numpy_iterator():
  print(x)


[10368 20736]
[ 5472 16128]
[ 29088 105120]
[120960]


In [ ]:
# Downloading Kaggle dataset
! pip install Kaggle

In [ ]:
 from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yousafhashmi","key":"dcd0d7799cb271ce399cc286939b3b03"}'}

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
import os

# Navigate to the target folder
os.chdir('/content/drive/MyDrive/Kaggle datasets')
!kaggle datasets download -d samuelcortinhas/cats-and-dogs-image-classification

 88% 57.0M/64.4M [00:01<00:00, 50.7MB/s]
100% 64.4M/64.4M [00:01<00:00, 50.6MB/s]


In [ ]:
#  Unzipping the dataset
!unzip $'/content/drive/MyDrive/Kaggle datasets/cats-and-dogs-image-classification.zip' -d '/content/drive/MyDrive/Kaggle datasets/Cats and Dogs'

In [ ]:
# Delete File COmmand
# import shutil
# shutil.rmtree('/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/test')


In [ ]:
# storing images filepaths where filepaths are in form of tensors
ds_images_path = tf.data.Dataset.list_files('/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/*/*', shuffle=False)
for x in ds_images_path:
  print(x.numpy())

b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_10.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_100.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_101.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_102.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_103.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_104.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_11.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_111.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_12.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_120.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats and Dogs/Cats and Dogs/cats/cat_123.jpg'
b'/content/drive/MyDrive/Kaggle datasets/Cats 

In [ ]:
ds_images_path = ds_images_path.shuffle(200)

In [ ]:
class_names = ['cat', 'dog']

In [ ]:
img_count = len(ds_images_path)

In [ ]:
img_count

557

In [ ]:
train_Ratio = int(img_count*0.8)
train_ds = ds_images_path.take(train_Ratio)
test_ds = ds_images_path.skip(train_Ratio)

In [ ]:
len(train_ds)

445

In [ ]:
len(test_ds)

112

In [ ]:
#  Creating a function to Retrieve labels from images paths
#  will be using the os seperator as file paths are tensor objects
def get_labels(file_path):
  import os
  return tf.strings.split(file_path, os.path.sep)[-2]


In [ ]:
# Creating function to read images from file_paths
def process_images(file_path):
  label = get_labels(file_path)
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, [128,128])

  return img, label

In [ ]:
for img,label in train_ds.map(process_images).take(5):
  print('Image:', img )
  print('Label:', label)

Image: tf.Tensor(
[[[ 25.379883   33.379883   35.379883 ]
  [ 27.876953   35.876953   37.876953 ]
  [ 24.624023   32.624023   34.624023 ]
  ...
  [ 52.625      48.625      39.625    ]
  [ 34.29297    36.29297    25.292969 ]
  [ 37.176758   38.176758   30.176758 ]]

 [[ 24.990234   32.990234   34.990234 ]
  [ 31.481445   39.481445   41.481445 ]
  [ 28.609375   36.609375   38.609375 ]
  ...
  [ 64.81641    60.816406   51.816406 ]
  [ 31.509766   33.509766   22.509766 ]
  [ 15.143555   16.143555    8.143555 ]]

 [[ 21.05371    29.05371    31.374023 ]
  [ 22.770508   30.770508   32.770508 ]
  [ 30.18457    38.18457    40.18457  ]
  ...
  [ 55.16504    53.125977   43.805664 ]
  [ 28.904297   30.904297   19.904297 ]
  [ 12.229492   13.229492    5.229492 ]]

 ...

 [[ 47.85547    40.964844   40.194336 ]
  [ 56.323242   42.323242   39.323242 ]
  [ 37.98242    30.982422   24.982422 ]
  ...
  [ 94.543945   81.543945   75.543945 ]
  [121.80469   108.80469   102.80469  ]
  [132.34473   115.34473  

In [ ]:
train_ds = train_ds.map(process_images)

In [ ]:
def scale(img, label):
  return img/255, label

In [ ]:
train_ds = train_ds.map(scale)